In [16]:
%load_ext autoreload
import sqlite3
import pandas as pd
from tabulate import tabulate
from utils import get_file_name_of_newest_database

name = get_file_name_of_newest_database()
conn = sqlite3.connect(name)
cursor = conn.cursor()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Oppgave 3
Her skal du kjøpe 9 voksenbilletter til forestillingen for Størst av alt er
kjærligheten 3. februar, hvor det er 9 ledige billetter og hvor stolene er på
samme rad. Stolene trenger ikke være ved siden av hverandre. Vi ønsker å få
summert hva det koster å kjøpe disse billettene,

In [17]:
# find all theaters by Oppstilling
%autoreload
from utils import chooseUserInput

query = '''
SELECT TeaterStykkeNavn, SalNavn 
FROM Oppstilling;
'''

theaters = cursor.execute(query).fetchall()
chosen_theater = chooseUserInput(theaters)
# chosen_theater = theaters[0]
play, sal = chosen_theater

# get all dates for the chosen theater
query = f'''
SELECT DISTINCT Dato, Tid
FROM Forestilling
WHERE TeaterStykkeNavn = "{play}";
'''

dates = cursor.execute(query).fetchall()
chosen_date = chooseUserInput(dates)
# chosen_date = dates[0]
date, time = chosen_date
# date , time = "2024-02-03", "19:00"

print(f'You have chosen the play "{play}" at the theater "{sal}" on the date "{date}" at "{time}"')

query = f'''
SELECT Nummer
FROM Forestilling
WHERE TeaterStykkeNavn = "{play}"
AND Dato = "{date}"
'''

ForestillingNummer = cursor.execute(query).fetchone()[0]
# print(ForestillingNummer)

query_stol = f'''
SELECT RadNr, StolNr, Område, SalNavn
FROM Stol
WHERE SalNavn = "{sal}"
AND NOT (RadNr, StolNr, Område, salNavn) IN (
    SELECT RadNr, StolNr, Område, salNavn
    FROM Billett
    WHERE ForestillingNummer = {ForestillingNummer}
    AND TeaterStykkeNavn = "{play}"
)
'''
available_seats = cursor.execute(query).fetchall()

query_distinct_rows_with_9_seats = f'''
SELECT DISTINCT Område, RadNr
FROM Stol
WHERE SalNavn = "{sal}"
AND NOT (RadNr, StolNr, Område, salNavn) IN (
    SELECT RadNr, StolNr, Område, salNavn
    FROM Billett
    WHERE ForestillingNummer = {ForestillingNummer}
    AND TeaterStykkeNavn = "{play}"
)
GROUP BY Område, RadNr  HAVING COUNT(*) >= 9;
'''

rows_with_9_seats = cursor.execute(query_distinct_rows_with_9_seats).fetchall()
# print(len(rows_with_9_seats))
# print(tabulate(rows_with_9_seats, headers=['RadNr', 'StolNr', 'Område', 'SalNavn']))

chooseRowAndArea = chooseUserInput(rows_with_9_seats, [0,1], "Velg område og rad for å se ledige seter")
# print("chooseRowAndArea",chooseRowAndArea)
område, rad = chooseRowAndArea

# finn alle stoler i valgt rad og område
query_seats_in_row = f'''
SELECT RadNr, StolNr, Område, SalNavn
FROM Stol
WHERE SalNavn = "{sal}"
AND RadNr = {rad}
AND Område = "{område}"
AND NOT (RadNr, StolNr, Område, salNavn) IN (
    SELECT RadNr, StolNr, Område, salNavn
    FROM Billett
    WHERE ForestillingNummer = {ForestillingNummer}
    AND TeaterStykkeNavn = "{play}"
)
'''
seats_in_row = cursor.execute(query_seats_in_row).fetchall()

# BilletType by teaterstykkenavn
query_billettype = f'''
SELECT Type, Pris 
FROM BilletType
WHERE TeaterStykkeNavn = "{play}" AND MinAntall <= 9
'''
billettype = cursor.execute(query_billettype).fetchall()


valgteStoler = []
billetType = []
while len(valgteStoler) < 9:
    valgteStoler.append(chooseUserInput(seats_in_row, [1], f'Velg stol {len(valgteStoler)+1} av 9 i rad {rad} og område {område}'))
    billettype.append(chooseUserInput(billettype, [0,1], f'Velg billettype for stol {len(valgteStoler)}'))
    seats_in_row = [x for x in seats_in_row if x not in valgteStoler]
print(valgteStoler)

You have chosen the play "Størst av alt er kjærligheten" at the theater "Gamle scene" on the date "2024-02-03" at "18:30"
[(1, 1, 'Balkong', 'Gamle scene'), (1, 2, 'Balkong', 'Gamle scene'), (1, 3, 'Balkong', 'Gamle scene'), (1, 4, 'Balkong', 'Gamle scene'), (1, 5, 'Balkong', 'Gamle scene'), (1, 6, 'Balkong', 'Gamle scene'), (1, 7, 'Balkong', 'Gamle scene'), (1, 8, 'Balkong', 'Gamle scene'), (1, 9, 'Balkong', 'Gamle scene')]


In [18]:
from datetime import datetime

cursor.execute("SELECT * FROM Kundeprofil")
customer = cursor.fetchone()
customerNumber = customer[0]

# get antall billettkjøp hvor ForestillingNummer, TeaterStykkeNavn
query = f'''
SELECT COUNT(*)
FROM Billettkjøp
WHERE ForestillingNummer = "{ForestillingNummer}"
AND TeaterStykkeNavn = "{play}";
'''
ticketNumber = cursor.execute(query).fetchone()[0] + 1


datetimeNow = datetime.now()
dateNow, timeNow = datetimeNow.strftime("%Y-%m-%d"), datetimeNow.time().strftime("%H:%M")

# billettkjøp
ticket_purchase = (
    play,
    ForestillingNummer,
    ticketNumber,
    customerNumber,
    dateNow,
    timeNow,
)
cursor.execute(
    "INSERT INTO Billettkjøp (TeaterStykkeNavn, ForestillingNummer, Nummer, Mobilnummer, dato, tid) VALUES (?, ?, ?, ?, ?, ?)",
    ticket_purchase,
)

    # RadNr INT NOT NULL,
    # StolNr INT NOT NULL,
    # Område VARCHAR(255) NOT NULL,
    # SalNavn VARCHAR(255) NOT NULL,
    # ForestillingNummer INTEGER NOT NULL,
    # BillettKjøpNummer INT NOT NULL,
    # Billettype VARCHAR(255) NOT NULL,
    # TeaterStykkeNavn VARCHAR(255) NOT NULL,
# kjøp billetter
for stol, billettype in zip(valgteStoler, billettype):
    cursor.execute(
        "INSERT INTO Billett (RadNr, StolNr, Område, SalNavn, ForestillingNummer, BillettKjøpNummer, Billettype, TeaterStykkeNavn) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
        (*stol, ForestillingNummer, ticketNumber, billettype[0], play),
    )


In [19]:
# get ticket for forestilling 6 and billettkjøp 2 and join them with ticket type and sum the price

query = f"""
SELECT SUM(BilletType.pris)
FROM Billett
JOIN BilletType
ON Billett.Billettype = BilletType.type
AND Billett.TeaterStykkeNavn = BilletType.TeaterStykkeNavn
WHERE Billett.ForestillingNummer = {ForestillingNummer}
AND Billett.BillettKjøpNummer = {ticketNumber}
AND Billett.TeaterStykkeNavn = "{play}"
"""
cursor.execute(query)
tickets = cursor.fetchall()
print(f'Billettene koster {tickets[0][0]} kr')

Billettene koster 2190 kr


In [20]:
# close the connection
conn.commit()
conn.close()